Para esta iteracion, calcularemos el conjunto de partes del conjunto de todas las columnas utiles para la prediccion y haremos un ranking para cada una de las cuatro metricas.

Primero buildeamos el codigo de C++ que hara la regresion lineal.

In [ ]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

Importamos las librearias necesarias.

In [1]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
import ModeloPrecio
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_error

En esta ocasion, estaremos utilizando el dataset de train el cual contiene los precios. Como se ve mas adelante, utilizaremos la tecnica de K-Fold para conseguir metricas menos sesgadas del dataset.

In [ ]:
df = pd.read_csv('../../data/train.csv')

Aqui se definen las columnas a utilizar, con las cuales calcularemos todas las combinaciones posibles.

In [ ]:
columnas_utilizadas = ['centroscomercialescercanos','escuelascercanas','piscina','usosmultiples','banos', 'habitaciones', 'antiguedad', 'metroscubiertos',
            'metrostotales', 'garages']

Por ultimo ejecutaremos nuestro modelo SinSegmentar el cual como dice su nombre, no utiliza ningun tipo de segmentacion. Otra caracteristica importante de nuestro modelo es que reemplaza cualquier informacion faltante de las columnas que utilice por el promedio de las mismas.

In [ ]:
KFOLD_K = 5
kf = KFold(n_splits=KFOLD_K, shuffle=True)

def powerset(s):
    conjunto_de_partes = []
    x = len(s)
    for i in range(1 << x):
        conjunto_de_partes.append([s[j] for j in range(x) if (i & (1 << j))])
    return conjunto_de_partes

result = {}
for picked_columns in powerset(columnas_utilizadas):
    rms = []
    rmsle = []
    mae = []
    r2 = []
    if picked_columns == []:
        continue

    for train_index, test_index in kf.split(df):
        df_train, df_test = df.loc[train_index], df.loc[test_index]
        
        mp = ModeloPrecio.SinSegmentar(df_train, picked_columns)
        df_predicted = mp.run(df_test)
        
        df_predicted['precio'] = df_predicted['precio'].copy().apply(lambda x: x if x > 0 else 0) 
        
        rms.append(mean_squared_error(df_test["precio"], df_predicted["precio"], squared=False))
        rmsle.append(mean_squared_log_error(df_test["precio"],  df_predicted['precio']))
        r2.append(r2_score(df_test["precio"], df_predicted["precio"]))
        mae.append(mean_absolute_error(df_test["precio"], df_predicted["precio"]))

    rms_result = np.sum(rms) / KFOLD_K 
    rmsle_result = np.sum(rmsle) / KFOLD_K 
    r2_result = np.sum(r2) / KFOLD_K 
    mae_result = np.sum(mae) / KFOLD_K 

    result['-'.join(picked_columns)] = np.array([rms_result, rmsle_result, r2_result, mae_result])

print(result)


Luego, este resultado es transformado en JSON y ordenado usando un script que nos devolvera otro JSON con 4 arrays de 5 elementos, uno para cada metrica junto con los 5 primeros.